In [1]:
import pandas as pd
import numpy as np
import os.path
import requests
import json

root_path = os.path.dirname(os.getcwd())

# Import food inspection data
inspections = pd.read_csv(os.path.join(root_path, "DATA/food_inspections.csv"))

In [2]:
# Create weather dataframe with entry for each inspection
weather = pd.DataFrame({
    "inspection_id": inspections.inspection_id,
    "date": inspections.inspection_date,
})

# Create list of weather data columns
features = [
    "precip_intensity",
    "temperature_max",
    "wind_speed",
    "humidity"
]

# Add empty columns
weather = pd.concat([weather, pd.DataFrame(columns=features)])

/home/russell/.local/lib/python2.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


In [3]:
# Check if weather data exists, import or start empty dataframe
if os.path.isfile(os.path.join(root_path, "DATA/weather.csv")):
    old_weather = pd.read_csv(os.path.join(root_path, "DATA/weather.csv"))
else:
    old_weather = pd.DataFrame(columns=weather.columns)

In [4]:
# Register for a free key at https://darksky.net/dev/register
KEY = "c30a95e4e9f7f2e90966361288918b9e"

In [5]:
# Check if weather record can be found in old data
# If not,request record from darksky.net
# All records pertain to a single location in central Chicago

limit_reached = False

def get_weather(group):
    global limit_reached
    
    date = group.iloc[0].date
    record = old_weather[old_weather.date == date]
    if (not record.empty) & (not record.isnull().values.any()):
        print(date, "record found")
        group.loc[:, features] = record.iloc[0][features].values
        
    elif not limit_reached:
        url = "https://api.darksky.net/forecast/%s/41.836944,-87.684722,%s" % (KEY, date)
        response = requests.get(url)
        
        if response.status_code == 200:
            print(date, "request successful")
            data = response.json()["daily"]["data"][0] 
            group[features] = [
                data["precipIntensity"],
                data["temperatureMax"],
                data["windSpeed"],
                data["humidity"]
            ]
        elif response.status_code == 403:
            print("request limit reached")
            limit_reached = True
        else:
            print(response)
    return group

In [6]:
# Time consuming
weather = weather.groupby(by="date").apply(get_weather)

('2010-01-04T00:00:00', 'request successful')
('2010-01-04T00:00:00', 'request successful')
('2010-01-05T00:00:00', 'request successful')
('2010-01-06T00:00:00', 'request successful')
('2010-01-07T00:00:00', 'request successful')
('2010-01-08T00:00:00', 'request successful')
('2010-01-11T00:00:00', 'request successful')
('2010-01-12T00:00:00', 'request successful')
('2010-01-13T00:00:00', 'request successful')
('2010-01-14T00:00:00', 'request successful')
('2010-01-15T00:00:00', 'request successful')
('2010-01-19T00:00:00', 'request successful')
('2010-01-20T00:00:00', 'request successful')
('2010-01-21T00:00:00', 'request successful')
('2010-01-22T00:00:00', 'request successful')
('2010-01-25T00:00:00', 'request successful')
('2010-01-26T00:00:00', 'request successful')
('2010-01-27T00:00:00', 'request successful')
('2010-01-28T00:00:00', 'request successful')
('2010-02-01T00:00:00', 'request successful')
('2010-02-02T00:00:00', 'request successful')
('2010-02-03T00:00:00', 'request s

KeyboardInterrupt: 

In [ ]:
# Check if process is complete
if weather.isnull().values.any():
    print("Process incomplete, records missing")
else:
    print("Process complete, all records obtained")

Darksky.net limits free accounts to 1000 calls per day. If records are missing, save the result and run the notebook from the top after waiting 24 hours, upgrading your account or obtaining a new API key.

In [ ]:
# Save result
weather.to_csv(os.path.join(root_path, "DATA/weather.csv"), index=False)